# Практичне 1. Словник


1. Текстові файли подаються на вхід в будь-якому форматі.
2. Розмір текстових файлів не менше 150 К.
3. Кількість текстових файлів не менше 10.
4. Словник термінів зберегти на диск.
5. Оцінити розмір колекції, загальну кількість слів в колекції та розмір словника.
4. Обгрунтувати структуру даних
5. Зробити оцінку складності алгоритму
6. Випробувати декілька форматів збереження словника (серіалізація словника, збереження в текстовий файл і т.д.) і порівняти результати.


Ініціалізуємо бібліотеку `nltk` для початку роботи.

In [5]:
from nltk.corpus import stopwords
from info_search.lab_1.nltk_utils import init_nltk

init_nltk()
stopwords.words("ukrainian")

[nltk_data] Downloading package punkt to /home/o.kyba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['а',
 'аби',
 'абиде',
 'абиким',
 'абикого',
 'абиколи',
 'абикому',
 'абикуди',
 'абихто',
 'абичий',
 'абичийого',
 'абичийому',
 'абичим',
 'абичию',
 'абичия',
 'абичиє',
 'абичиєму',
 'абичиєю',
 'абичиєї',
 'абичиї',
 'абичиїй',
 'абичиїм',
 'абичиїми',
 'абичиїх',
 'абичого',
 'абичому',
 'абищо',
 'абияка',
 'абияке',
 'абиякий',
 'абияким',
 'абиякими',
 'абияких',
 'абиякого',
 'абиякому',
 'абиякою',
 'абиякої',
 'абияку',
 'абиякі',
 'абиякій',
 'абиякім',
 'або',
 'абощо',
 'авжеж',
 'авось',
 'ага',
 'ад',
 'адже',
 'аж',
 'ажень',
 'аз',
 'ай',
 'але',
 'ало',
 'амінь',
 'ант',
 'ану',
 'ані',
 'аніде',
 'аніж',
 'анізащо',
 'аніким',
 'анікого',
 'анікогісінько',
 'аніколи',
 'анікому',
 'аніскільки',
 'аніхто',
 'анічим',
 'анічого',
 'анічогісінько',
 'анічому',
 'аніщо',
 'аніяка',
 'аніяке',
 'аніякий',
 'аніяким',
 'аніякими',
 'аніяких',
 'аніякого',
 'аніякому',
 'аніякою',
 'аніякої',
 'аніяку',
 'аніякі',
 'аніякій',
 'аніякім',
 'аніякісенька',
 'аніякісеньк

## Текстові файли

Для обробки було обрано книжки у форматі `EBUB`, що знаходяться у директорії `books`.

In [3]:
books = (
    "books/451-za-farengeytom.epub",
    "books/1984.epub",
    "books/atlant-rozpraviv-pliechi-1.epub",
    "books/atlant-rozpraviv-pliechi-2.epub",
    "books/atlant-rozpraviv-pliechi-3.epub",
    "books/haksli-oldos-prekrasnyy-novyy-svit.epub",
    "books/kulbabove-vino.epub",
    "books/na-zakhidnomu-fronti-bez-zmin.epub",
    "books/proshchavai-zbroie.epub",
    "books/sapients-istoriya-lyudstva.epub",
)

## Побудуємо словник

1. Для читання книжок використовуємо клас `info_search.lab_1.readers.EpubReader`
2. Для токінізації тексту використовуємо `info_search.lab_1.preprocessing.WordsTokenizer`. Цей клас розбиває текст на речення та слова за допомогою бібліотеки: `nltk.sent_tokenize(text)` та `nltk.word_tokenize(sentence)`, а також приводить слова до нижнього регістру.
3. Для нормалізації слів використовуємо клас `info_search.lab_1.preprocessing.WordsNormalizer`. Він вкиконю легматизацію слів української мови завдяки бібілотеці `pymorphy3`.

In [7]:
from pymorphy3 import MorphAnalyzer

from info_search.lab_1.lexicon import Lexicon
from info_search.lab_1.parsers import parse_terms
from info_search.lab_1.readers import EpubReader
from info_search.lab_1.preprocessing import WordsNormalizer, WordsTokenizer

reader = EpubReader(books)
tokenizer = WordsTokenizer(stopwords.words("ukrainian"))
normalizer = WordsNormalizer(MorphAnalyzer(lang="uk"))
lexicon = Lexicon()

for doc_name, term in parse_terms(reader, tokenizer, normalizer):
    doc_id, term_id = lexicon.add_term(doc_name, term)

  0%|          | 0/10 [00:00<?, ?it/s]/home/o.kyba/PycharmProjects/info_search/venv/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
100%|██████████| 10/10 [00:50<00:00,  5.03s/it]


## Про внутрішну будову Lexicon

В цій роботі словник відповідальний не тільки за збереження усіх термінів, а й за видачу та збереженню інформації про ідентифікатори термінів та документів.

Структура даних для термінв - це `dict` (`HashMap`), ключем якого є термін, а значеннями пара - ідентифікатор та чатота появи терміну в колекції. Таким чином ми отримуємо константний час на знаходження ідентифікатора терміну та його частоти - `O(1)`. З мінусів: можливі колізії у `dict`, що збільшує час вставки та знаходження ідентифікатора терміну, збереження хешів у пам'яті (закрита адресація хеш-таблиці у `python`) та місце у пам'яті, що не використовується у хеш-таблиці, але має бути зайнято нею.

Для документів використовується структура даних `OrderedSet`, що реалізується за допомогою `dict` та `list` (`ArrayList`). Така незвична структура даних дає змогу додавати документи, зберігаючи їх унікальність, за константний час `O(1)` та знаходити документ по ідентифікатору теж за константний час `O(1)`, виконуючи діставання ім'я документа по його ідентифиіквтору (`list[i]`). Недолік цієї структури даних - це витрата пам'яті і на `dict`, і на `list`, всі нелоліки `dict` та перебудова `list` кожного разу, коли його внутрішній масив заповнюється.

## Оцінюємо розміри

In [12]:
from info_search.lab_1.size import get_folder_size

collection_size = get_folder_size("./books")
totlal_terms_count = lexicon.total_terms_count
lexicon_size = len(lexicon.terms)

print(f"Розмір колекції: {collection_size}")
print(f"Загальна кількість слів в колекції: {totlal_terms_count}")
print(f"Розмір словника: {lexicon_size}")

Розмір колекції: 7.503994941711426 MB
Загальна кількість слів в колекції: 449481
Розмір словника: 36447


## Збереження словника на диск

Було спробувано зберігати словник у кількох форматах: `pickle`, `json` та `protobuf`.

У цій роботі для словника відбувається збереження не тільки самих термінів та їх частоти, а й інформація про відповідність ідентифікаторів.

* Найшвидшим методом серіалізації є `pickle`, бо він є вбудованим рішенням для мови програмування `python`.
* Найбільший файл продукує `JSON`, бо цей формат не є економним. Але через це саме цей формат є найкращочитаємим для людини, що може бути корисно при відладках додатку.
* Найменший файл продукує `Protobuf`, бо він є оптимізованим якраз під цю задачу. Але через те, що нам доводиться кожного разу перетворювати `dict` у список об'єків, цей метод відпрацьовує найдовше.

In [16]:
import os
import time

print(
    f"Словник до операцій: "
    f"unique_terms_count={len(lexicon.terms)}. "
    f"total_terms_count={lexicon.total_terms_count}"
)

start_time = time.time()

with open("./data/lexicon.pkl", "wb") as file:
    file.write(lexicon.to_pickle())

with open("./data/lexicon.pkl", "rb") as file:
    lexicon_from_pickle = Lexicon.from_pickle(file.read())

print(
    f"Словник після збереження в форматі pickle: "
    f"unique_terms_count={len(lexicon_from_pickle.terms)}. "
    f"total_terms_count={lexicon_from_pickle.total_terms_count}, "
    f"time={time.time() - start_time}, "
    f"file_size={os.path.getsize('./data/lexicon.pkl')}"
)

start_time = time.time()

with open("./data/lexicon.json", "w") as file:
    file.write(lexicon.to_json())

with open("./data/lexicon.json", "r") as file:
    lexicon_from_json = Lexicon.from_json(file.read())

print(
    f"Слоаник після збереження у форматі JSON: "
    f"unique_terms_count={len(lexicon_from_json.terms)}. "
    f"total_terms_count={lexicon_from_json.total_terms_count}, "
    f"time={time.time() - start_time}, "
    f"file_size={os.path.getsize('./data/lexicon.json')}"
)

start_time = time.time()

with open("./data/lexicon.protobuf", "wb") as file:
    file.write(lexicon.to_proto())

with open("./data/lexicon.protobuf", "rb") as file:
    lexicon_from_proto = Lexicon.from_proto(file.read())

print(
    f"Словник після збереження у форматі Protobuf: "
    f"unique_terms_count={len(lexicon_from_proto.terms)}. "
    f"total_terms_count={lexicon_from_proto.total_terms_count}, "
    f"time={time.time() - start_time}, "
    f"file_size={os.path.getsize('./data/lexicon.protobuf')}"
)

Словник до операцій: unique_terms_count=36447. total_terms_count=449481
Словник після збереження в форматі pickle: unique_terms_count=36447. total_terms_count=449481, time=0.09282183647155762, file_size=1092322
Слоаник після збереження у форматі JSON: unique_terms_count=36447. total_terms_count=449481, time=0.21279191970825195, file_size=2504575
Словник після збереження у форматі Protobuf: unique_terms_count=36447. total_terms_count=449481, time=0.21961569786071777, file_size=1002892
